# Core Keras API

In [98]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, Sequential, optimizers, losses, metrics
from tensorflow.keras.layers import Layer, Dense

In [99]:
class VanillaDense(Layer):

    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,), initializer="zeros")

    def call(self, inputs):
        
        y = tf.matmul(inputs, self.W) + self.b
        
        if self.activation is not None:
            y = self.activation(y)
        
        return y

In [100]:
vanilla_dense = VanillaDense(units=32, activation=tf.nn.relu)
inputs = tf.ones(shape=(2, 784))
outputs = vanilla_dense(inputs)
print(outputs.shape)

(2, 32)


#### Automatic shape inference: Building layers on the fly

In [101]:
model = Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
])
model = Sequential([Dense(1)])

### The "compile" step: Configuring the learning process

In [102]:
model.compile(optimizer="rmsprop", loss="mean_squared_error", metrics=["accuracy"])

**Calling `fit()` with NumPy data**

In [103]:
history = model.fit(
    inputs,
    outputs,
    epochs=5,
    batch_size=128
)

Epoch 1/5
1/1 [==============================] - 0s 207ms/step - loss: 2.6054 - accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 4ms/step - loss: 1.7744 - accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 3ms/step - loss: 0.8528 - accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 0s 3ms/step - loss: 0.6711 - accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 5ms/step - loss: 0.6352 - accuracy: 0.5000


In [104]:
history.history

{'loss': [2.605405330657959,
  1.774379014968872,
  0.8528051376342773,
  0.6710835695266724,
  0.635200023651123],
 'accuracy': [0.5, 0.0, 0.5, 0.0, 0.5]}

### Monitoring loss and metrics on validation data

In [115]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(mean=[0, 3], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(mean=[3, 0], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)

inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"), 
                     np.ones((num_samples_per_class, 1), dtype="float32")))

In [116]:
indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs)) # 30 percent

val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]

training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

In [117]:
model = Sequential([Dense(1)])
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.1),
              loss=losses.MeanSquaredError(),
              metrics=[metrics.BinaryAccuracy()])

In [118]:
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)

Epoch 1/5
88/88 [==============================] - 1s 4ms/step - loss: 0.1061 - binary_accuracy: 0.9593 - val_loss: 0.0361 - val_binary_accuracy: 0.9983
Epoch 2/5
88/88 [==============================] - 0s 4ms/step - loss: 0.0718 - binary_accuracy: 0.9586 - val_loss: 0.1259 - val_binary_accuracy: 0.9933
Epoch 3/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0650 - binary_accuracy: 0.9686 - val_loss: 0.0919 - val_binary_accuracy: 0.9200
Epoch 4/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0681 - binary_accuracy: 0.9650 - val_loss: 0.0911 - val_binary_accuracy: 0.9700
Epoch 5/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0738 - binary_accuracy: 0.9550 - val_loss: 0.3337 - val_binary_accuracy: 0.6967


### Inference: Using a model after training

In [119]:
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

5/5 [==============================] - 0s 1ms/step
[[0.47338504]
 [1.6877916 ]
 [1.7080666 ]
 [1.798213  ]
 [0.84774727]
 [1.6445491 ]
 [0.31429505]
 [0.81216204]
 [0.09219342]
 [0.27464375]]
